In [1]:
import pathlib
import os
from pathlib import Path
import sys
 
if sys.argv:
    sys.path.insert(0, str(pathlib.Path(os.path.dirname(os.path.abspath(""))).resolve()))
%load_ext autoreload
%autoreload 2

In [2]:
from src.utils import get_paths_train_test, train_test_model
from src.factory import EFINUpliftModelFactory
from src.models.NNUpliftModeling.EFINUpliftModel import EFINUpliftModel
import pandas as pd

/home/ogrobertino/test_env/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
Failed to import duecredit due to No module named 'duecredit'


In [3]:
ds_name = 'lazada_v2'
features_percent = 100
train_path, test_path = get_paths_train_test(ds_name=ds_name, features_percent=features_percent)

In [4]:
import pandas as pd
ds = pd.read_csv(train_path, sep='\t')

In [5]:
input_size = len(ds.columns) - 2
input_size

83

In [6]:
params = {
    'input_dim': input_size, 
    'hc_dim': 16,    
    'hu_dim': 8,      
    'act_type': 'elu',    
    
    'batch_size': 128,
    'epochs': 1,
    'early_stopping_patience': 2,
    
    'optimizer': 
        {
            'name': 'Adam',
            'lr': 0.005,
            'weight_decay': 0.001
        },
    
    # Настройки планировщика
    'scheduler': {
        'name': 'ReduceLROnPlateau',
        'patience': 3,
        'factor': 0.5
    }
}

In [7]:
configs = EFINUpliftModel.generate_config(count=4, params = params)

In [8]:
factory = EFINUpliftModelFactory
config = configs[0]
batch_size=32
max_size=100000

In [9]:
config

{'batch_size': 128,
 'epochs': 1,
 'early_stopping_patience': 2,
 'optimizer': {'name': 'Adam', 'lr': 0.005, 'weight_decay': 0.001},
 'scheduler': {'name': 'ReduceLROnPlateau', 'patience': 3, 'factor': 0.5},
 'use_gpu': True,
 'num_workers': 0,
 'inference_batch_size': 32,
 'input_dim': 83,
 'hc_dim': 16,
 'hu_dim': 8,
 'is_self': True,
 'act_type': 'elu',
 'learning_rate': 0.01}

In [10]:
model, train, test = factory.create(config, train_path, test_path)

/home/ogrobertino/test_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [11]:
model.num_params()

12258

In [12]:
model

In [11]:
# from torch.utils.data import DataLoader
# dl = DataLoader(
#     train, 
#     batch_size=32, 
#     shuffle=True,

#     num_workers=0
# )

In [12]:
# kek = model.predict(train)

In [13]:
# pd.DataFrame(kek['uplift'])

In [14]:
# q = model.predict(train)

In [15]:
# q['df'][q['df']['treatment'] == 1]

In [13]:
train_test_model(ds_name='lazada_v2', features_percent=100, factory=EFINUpliftModelFactory, config=configs[0], NN=True)   

/home/ogrobertino/test_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/1 - 10.0% - Loss: 0.4218
Epoch 1/1 - 20.0% - Loss: 0.4004
Epoch 1/1 - 30.0% - Loss: 0.3911
Epoch 1/1 - 40.0% - Loss: 0.3854
Epoch 1/1 - 50.0% - Loss: 0.3795
Epoch 1/1 - 60.0% - Loss: 0.3752
Epoch 1/1 - 70.0% - Loss: 0.3719
Epoch 1/1 - 79.9% - Loss: 0.3672
Epoch 1/1 - 89.9% - Loss: 0.3624
Epoch 1/1 - 99.9% - Loss: 0.3565
Validation after epoch
Epoch 1/1, Train Loss: 0.3565, Val Loss: 0.3030, Val AUUC: -0.0455, LR: 0.005000
Модель, предсказания и конфиг сохранены в директории exps2/lazada_v2/100/14
Эксперимент сохранен в таблице /home/ogrobertino/UpliftModelingResearch/exps2/stats.tsv


In [ ]:
for i in tqdm(range(len(configs)), desc="Training models"):
    train_test_model(ds_name='lazada', features_percent=100, factory=EFINUpliftModelFactory, config=configs[i])   

In [16]:
from src.global_params import BASE_PATH
path = pd.read_csv(BASE_PATH + "/exps2/stats.tsv", sep='\t').Path.iloc[-1]
model = EFINUpliftModelFactory(from_load=True, path = str(BASE_PATH) + "/" + path)

/home/ogrobertino/UpliftModelingResearch/src/models/NNUpliftModeling/INNUpliftModeling.py:452: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, ma